In [17]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [2]:
zero_noise = pd.read_csv("data/zero_noise.csv")
low_noise = pd.read_csv("data/low_noise.csv")
high_noise = pd.read_csv("data/high_noise.csv")
low_noise.drop(columns=["data_type"], inplace=True)
high_noise.drop(columns=["data_type"], inplace=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.manual_seed(42)
# torch.set_default_dtype(torch.float32)
# torch.set_default_device(device)

In [3]:
def encode(v, class_values):
    return class_values.index(v)

def encode_target(v, class_values):
    return class_values.index(v)

class_values_zero = list(zero_noise.era.unique())
zero_noise["era_label"] = zero_noise["era"].apply(encode, args=(class_values_zero,))

class_values_low = list(low_noise.era.unique())
class_values_low_target_5 = list(low_noise.target_5_val.unique())
class_values_low_target_10 = list(low_noise.target_10_val.unique())
low_noise["era_label"] = low_noise["era"].apply(encode, args=(class_values_low,))
low_noise["target_5_val_label"] = low_noise["target_5_val"].apply(encode_target, args=(class_values_low_target_5,))
low_noise["target_10_val_label"] = low_noise["target_10_val"].apply(encode_target, args=(class_values_low_target_10,))

class_values_high = list(high_noise.era.unique())
class_values_high_target_5 = list(high_noise.target_5_val.unique())
class_values_high_target_10 = list(high_noise.target_10_val.unique())
high_noise["era_label"] = high_noise["era"].apply(encode, args=(class_values_high,))
high_noise["target_5_val_label"] = high_noise["target_5_val"].apply(encode_target, args=(class_values_high_target_5,))
high_noise["target_10_val_label"] = high_noise["target_10_val"].apply(encode_target, args=(class_values_high_target_10,))

dataset = high_noise
target_column = "target_10_val_label"

In [4]:
class CustomDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return (
            torch.tensor(self.X[index], dtype=torch.float32, device=device),
            torch.tensor(self.y[index], dtype=torch.long, device=device),
        )

In [5]:
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))

dataset = dataset.sample(frac=1).reset_index(drop=True)
train_dataset = dataset.iloc[:train_size]
val_dataset = dataset.iloc[train_size:train_size+val_size]
test_dataset = dataset.iloc[train_size+val_size:]

# train_datasets = []
# for i in range(5):
#     train_datasets.append(train_dataset.sample(frac=0.5, replace=True))

train_X = train_dataset.iloc[:, :-8].values
train_y = train_dataset.loc[:, target_column].values

# train_X = []
# train_y = []
# for i in range(5):
#     train_X.append(train_datasets[i].iloc[:, :-6].values)
#     train_y.append(train_datasets[i].iloc[:, -1].values)

val_X = val_dataset.iloc[:, :-8].values
val_y = val_dataset.loc[:, target_column].values

test_X = test_dataset.iloc[:, :-8].values
test_y = test_dataset.loc[:, target_column].values

In [6]:
print(np.unique(train_y, return_counts=True))
print(np.unique(val_y, return_counts=True))
print(np.unique(test_y, return_counts=True))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int64), array([16527, 16715, 16547, 16414, 16836, 16632, 16848, 16669, 16744,
       16594, 16572, 16582], dtype=int64))
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int64), array([2045, 2038, 2159, 2095, 1967, 2112, 2057, 2042, 2136, 2160, 2032,
       2117], dtype=int64))
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int64), array([2072, 2104, 2076, 2136, 2101, 2068, 2029, 2021, 2071, 2107, 2069,
       2106], dtype=int64))


In [7]:
train_dataset = CustomDataset(train_X, train_y)
# for i in range(5):
#     train_datasets[i] = CustomDataset(train_X[i], train_y[i])

val_dataset = CustomDataset(val_X, val_y)
test_dataset = CustomDataset(test_X, test_y)

# train_dataloaders = []
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# for i in range(5):
#     train_dataloaders.append(DataLoader(train_datasets[i], batch_size=batch_size, shuffle=True))
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [13]:
class MLP(nn.Module):
    def __init__(self, layers, autoencoder):
        super().__init__()
        self.layers = layers
        self.autoencoder = autoencoder
        self.linear = nn.Linear(12, 1)

    def forward(self, x):
        # x = self.autoencoder.encoder(x)
        # x = self.autoencoder.decoder(x)
        # for layer in self.layers:
        # x = layer(x)
        # return x

        for layer in self.layers[:-1]:
            x = layer(x)

        logits = self.layers[-1](x).to(torch.float64).to(device)
        # print("logits shape", logits.shape)
        x1 = self.linear(x)
        tau = torch.sigmoid(x1).to(torch.float64).to(device)
        # print("tau shape", tau.shape)

        return logits, tau


class NoiseAttentionLoss(nn.Module):
    def __init__(self, lambda_=1.0):
        super(NoiseAttentionLoss, self).__init__()
        self.lambda_ = lambda_

    def forward(self, logits, y, tau):
        y = F.one_hot(y, 12).to(torch.float64).to(device)
        perceptual = tau.squeeze(1) * (logits.t() - y.t()) + y.t()
        attention_term = torch.matmul(y, torch.log(perceptual + 1e-8))
        attention_term = attention_term.diag()
        boost_term = torch.log(tau + 1e-8) * self.lambda_

        attention_term = -torch.mean(attention_term)
        boost_term = -torch.mean(boost_term)

        return attention_term + boost_term


class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 48),
            nn.ReLU(),
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            # nn.ReLU(),
            # nn.Linear(64, encoding_dim),
        )
        self.decoder = nn.Sequential(
            # nn.Linear(encoding_dim, 16),
            # nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 48),
            nn.ReLU(),
            nn.Linear(48, input_dim),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


layers = nn.ModuleList(
    [
        nn.Linear(24, 64),
        nn.ReLU(),
        # nn.Dropout(0.1),
        # nn.Linear(64, 256),
        # nn.ReLU(),
        # nn.Dropout(0.1),
        nn.Linear(64, 128),
        nn.ReLU(),
        # nn.Dropout(0.1),
        # nn.Linear(96, 32),
        # nn.ReLU(),
        nn.Linear(128, 32),
        nn.ReLU(),
        nn.Linear(32, 12),
        nn.Softmax(dim=1),
    ]
)

input_dim = 24
encoding_dim = 8
autoencoder = Autoencoder(input_dim, encoding_dim).to(device)
criterion_auto = nn.MSELoss()
learning_rate_auto = 0.001

optimizer_auto = torch.optim.Adam(autoencoder.parameters(), lr=learning_rate_auto)
epochs_auto = 100

model = MLP(layers, autoencoder).to(device)
# criterion = nn.NLLLoss()
criterion = NoiseAttentionLoss(lambda_=50).to(device)
learning_rate = 0.0025
weight_decay = 0.0001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
epochs = 50

In [14]:
def train(model, train_dataloader, val_dataloader, epochs, criterion, optimizer):
    train_losses = []
    val_losses = []
    train_acc = []
    val_acc = []

    for epoch in tqdm(range(epochs)):
        running_loss_train = 0.0
        running_loss_val = 0.0
        running_corrects_train = 0.0
        running_corrects_val = 0.0
        train_loss = []
        accuracy_train = []
        
        # for i in range(5):
            # models[i].train()
        for _, (inputs, labels) in enumerate(train_dataloader):

            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            # encoded_inputs = autoencoder.encoder(inputs)
            # decoded_inputs = autoencoder.decoder(encoded_inputs)
            outputs, tau = model(inputs)
            loss = criterion(outputs, labels, tau)
            # outputs = model(inputs)
            # loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss_train += loss.item()
            running_corrects_train += torch.sum(torch.argmax(outputs, dim=1) == labels) / len(labels)

        # train_loss[i] = running_loss_train[i] / len(train_dataloaders[0])
        # accuracy_train[i] = running_corrects_train[i] / len(train_dataloaders[0])

        # train_loss = np.mean(train_loss)
        # accuracy_train = np.mean(accuracy_train)

        train_loss = running_loss_train / len(train_dataloader)
        accuracy_train = running_corrects_train / len(train_dataloader)
        train_losses.append(train_loss)
        train_acc.append(accuracy_train)

        # combined_model = combine_models([model for _ in range(5)])

        # if epoch % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs} - Training loss: {train_loss}")
        print(f"Epoch {epoch+1}/{epochs} - Training accuracy: {accuracy_train}")
        print("-----------------------------------------------")

        # val_model = models[val_model_index]
        # val_model.eval()
        model.eval()
        with torch.no_grad():
            for _, (val_inputs, val_labels) in enumerate(val_dataloader):
                val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)
                val_outputs, val_tau = model(val_inputs)
                loss = criterion(val_outputs, val_labels, val_tau)
                running_loss_val += loss.item()
                running_corrects_val += torch.sum(torch.argmax(val_outputs, dim=1) == val_labels) / len(val_labels)

            val_loss = running_loss_val / len(val_dataloader)
            accuracy_val = running_corrects_val / len(val_dataloader)
            val_losses.append(val_loss)
            val_acc.append(accuracy_val)

        # if epoch % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs} - Validation loss: {val_loss}")
        print(f"Epoch {epoch+1}/{epochs} - Validation accuracy: {accuracy_val}")
        print("===============================================")

    return train_losses, val_losses, train_acc, val_acc

In [15]:
train_losses, val_losses, train_acc, val_acc = train(model, train_dataloader, val_dataloader, epochs, criterion, optimizer)

  0%|          | 0/250 [00:00<?, ?it/s]

Epoch 1/250 - Training loss: 1.8630083905102461
Epoch 1/250 - Training accuracy: 0.37116387486457825
-----------------------------------------------
Epoch 1/250 - Validation loss: 1.4392076647847425
Epoch 1/250 - Validation accuracy: 0.46830931305885315
Epoch 2/250 - Training loss: 1.3850391689304027
Epoch 2/250 - Training accuracy: 0.4823117256164551
-----------------------------------------------
Epoch 2/250 - Validation loss: 1.3400257211647768
Epoch 2/250 - Validation accuracy: 0.4951923191547394
Epoch 3/250 - Training loss: 1.3113322010542805
Epoch 3/250 - Training accuracy: 0.5069711804389954
-----------------------------------------------
Epoch 3/250 - Validation loss: 1.2778847669458477
Epoch 3/250 - Validation accuracy: 0.5175080299377441
Epoch 4/250 - Training loss: 1.272865087750773
Epoch 4/250 - Training accuracy: 0.5204226970672607
-----------------------------------------------
Epoch 4/250 - Validation loss: 1.2676464748419474
Epoch 4/250 - Validation accuracy: 0.52011221

KeyboardInterrupt: 

In [18]:
model.eval()
with torch.no_grad():
    running_corrects_test = 0.0
    for _, (test_inputs, test_labels) in enumerate(test_dataloader):
        test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)
        test_outputs, test_tau = model(test_inputs)
        running_corrects_test += torch.sum(torch.argmax(test_outputs, dim=1) == test_labels) / len(test_labels)

    accuracy_test = running_corrects_test / len(test_dataloader)
    print(f"Test accuracy: {accuracy_test}")

Test accuracy: 0.5562500357627869


In [16]:
plt.figure(figsize=(10, 5))
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy Curve')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

NameError: name 'train_losses' is not defined

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss Curve')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [19]:
# torch.save(model.state_dict(), "high_noise_era.pth")